In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None

In [2]:
df_billboard = pd.read_csv(
    'data/Billboard-Weekly-Songs-With-Song-And-Artist-Spotify-Popularity.csv')
df_dataset = pd.read_csv('data/dataset.csv').drop(columns = 'Unnamed: 0')
df_artist_genres = pd.read_csv('data/artists_genres.csv')

In [3]:
df_billboard = df_billboard[df_billboard['date'] >= '1958-08-08']
df_billboard = df_billboard[df_billboard['date'] < '1996-01-01']

In [4]:
df_billboard = df_billboard.drop(columns = ['Spotify_Popularity', 'Artist_Popularity', 'Weeks on chart', 'year'])

In [5]:
df_billboard = df_billboard.rename(columns = {'Rank':'rank', 'Title':'title_billboard', 
                                              'Artist':'artist_billboard'})

In [6]:
len(df_billboard)

177700

In [7]:
df_dataset = df_dataset.drop(columns = ['Spotify_Popularity', 'Artist_Popularity', 'danceability',
                                       'speechiness', 'acousticness', 'instrumentalness', 'liveness', 
                                        'Weeks on chart', 'year'])

In [8]:
df_dataset = df_dataset[['track_id', 'Title', 'Artist', 'track_title', 'track_artist', 'key', 'time_signature',
                        'mode', 'tempo', 'loudness', 'duration_ms', 'energy', 'valence']]

In [9]:
df_dataset = df_dataset.rename(columns = {'Title':'title_billboard', 'Artist':'artist_billboard',
                                         'track_title':'title_spotify', 'track_artist':'artist_spotify'})

In [10]:
df_dataset = df_dataset.dropna()

In [11]:
df_genres = df_artist_genres[['Artist', 'genre']]
df_genres = df_genres.rename(columns = {'Artist': 'artist_billboard'})
df_genres = df_genres.drop_duplicates(['artist_billboard'])

In [12]:
len(df_dataset)

27065

In [13]:
len(df_genres)

5361

In [14]:
df_dataset = df_dataset.merge(df_genres, on='artist_billboard', how = 'left')

In [15]:
df_dataset = df_dataset.rename(columns = {'genre': 'genre_list'})

In [16]:
df_dataset = df_dataset.reindex()

In [17]:
df_dataset['weeks_list'] = np.nan
df_dataset['ranks_list'] = np.nan
df_dataset['highest_rank'] = 100

In [18]:
def serie_string_to_list(serie, limit = 1):
    
    serie_ = serie
    
    for ind, text in serie.iteritems():
        list_ = []
 
        split = re.split('[[\],\'"]', str(text))
            
        for item in split:
            if len(item) > limit:
                list_.append(item)
                
        serie_[ind] = list_
        
    return serie_

In [19]:
df_dataset[df_dataset['genre_list'] == np.nan] = '[]'

In [20]:
df_dataset['genre_list'] = serie_string_to_list(df_dataset['genre_list'])

In [21]:
for ind, row in df_dataset.iterrows():
    
    entries = df_billboard[(df_billboard['title_billboard'] == row['title_billboard']) &
                           (df_billboard['artist_billboard'] == row['artist_billboard'])]
    
    df_dataset['weeks_list'][ind] = str(entries['date'].values.tolist())
    df_dataset['ranks_list'][ind] = str(entries['rank'].values.tolist())
    
    if ind%1000 == 0:
        print(str(100*ind/len(df_dataset))[:5] + '%')
    

0.0%
3.694%
7.389%
11.08%
14.77%
18.47%
22.16%
25.86%
29.55%
33.25%
36.94%
40.64%
44.33%
48.03%
51.72%
55.42%
59.11%
62.81%
66.50%
70.20%
73.89%
77.59%
81.28%
84.98%
88.67%
92.37%
96.06%
99.75%


In [26]:
df_dataset = df_dataset[df_dataset['weeks_list'] != np.nan]

In [27]:
df_dataset['weeks_list'] = serie_string_to_list(df_dataset['weeks_list'])
df_dataset['ranks_list'] = serie_string_to_list(df_dataset['ranks_list'], limit = 0)

In [28]:
for ind, row in df_dataset.iterrows():
    
    if len(row['ranks_list']) > 0:
        df_dataset['highest_rank'][ind] = sorted(row['ranks_list'])[0]

In [40]:
df_dataset = df_dataset[df_dataset['weeks_list'].astype('str') != '[]']
df_dataset = df_dataset[df_dataset['ranks_list'].astype('str') != '[]']

In [41]:
len(df_dataset)

19098

In [42]:
df_dataset.to_json('json/corpus.json')

In [43]:
df_dataset.to_csv('data/corpus.csv')